# NELL Test Tool

## Setup

In [ ]:
from IPython.display import clear_output

%pip install selenium 
clear_output(wait=True)

%pip install webdriver_manager 
clear_output(wait=True)

%pip install beautifulsoup4
clear_output(wait=True)

%pip install pandas 
clear_output(wait=True)

%pip install ipywidgets
clear_output(wait=True)

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

from bs4 import BeautifulSoup
from collections import defaultdict
from xml.sax.saxutils import quoteattr
from IPython.display import display
from ipywidgets import HTML
import pandas
import ipywidgets as widgets
import time

import http.server
import socketserver
import threading

clear_output()
print("All packages installed")

## Source Code

### Selenium Bindings

In [ ]:

prefs = {"profile.default_content_setting_values.notifications": 1}
chrome_options = Options()
#chrome_options.add_argument("--headless")
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.set_capability('goog:loggingPrefs', {'browser': 'ALL'})

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.quit()


def is_page_loaded():
    global test_window
    try: return test_window.execute_script("return document.readyState;") == "complete"
    except: return False


def is_logging_instrumented():
    global test_window
    try: return test_window.execute_script("return window.hasEventListeners === true;")
    except: return False


def new_driver(url, wait=0):
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)
    time.sleep(wait)
    return driver


def highlight_element(driver, selector):
    global test_window
    if driver is None:
        driver = test_window
        
    script = f"""
    var element = document.evaluate('{selector}', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
    if (element) {{
        var originalStyle = element.style.border;
        var count = 0;
        var interval = setInterval(function() {{
            count += 1;
            element.style.border = count % 2 ? '3px solid red' : '';
            if (count > 9) {{
                clearInterval(interval);
                element.style.border = originalStyle;
            }}
        }}, 500);
    }}
    """
    try:
        driver.execute_script(script)
    except Exception as e:
        print("Error while highlighting the element:", e)


# generate keys
def generate_key(tag, element, counter):
    base = {"input": "txt", "button": "btn", "a": "lnk", "img": "img"}
    prefix = base.get(tag, "elem")

    parts = [prefix]
    
    txt = element.get_text(strip=True)
    if txt:
        parts.append(txt.replace(" ", "_"))
        return key_builder(parts, counter)
    
    if element.get('name'):
        parts.append(element['name'])
        return key_builder(parts, counter)
    
    if element.get('aria-label'):
        parts.append(element['aria-label'].replace(" ", "_"))
        return key_builder(parts, counter)

    if element.get('id'):
        parts.append(element['id'])
        return key_builder(parts, counter)
    
    if element.get('data-test-id'):
        parts.append(element['data-test-id'])
        return key_builder(parts, counter)
    
    return key_builder(parts, counter)


def key_builder(parts, counter):
    key = '_'.join(filter(None, parts)).lower().replace("-", "_")
    counter[key] += 1
    return key if counter[key] == 1 else f"{key}_{counter[key]}"


# Generate Selectors
def generate_selector(element, driver):
    selectors = []

    for attr in ['id', 'name', 'placeholder', 'aria-label', 'data-test-id', 'alt']:
        if element.get(attr):
            value = quoteattr(element[attr])
            selectors.append(f"@{attr}={value}")


    if selectors:
        xpath_selector = f"//{element.name}[{' and '.join(selectors)}]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector


    txt = element.get_text(strip=True)
    if txt:
        xpath_selector = f"//{element.name}[contains(text(), {quoteattr(txt)})]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector


    if element.get('class'):
        classes = '.'.join(element.get('class'))
        xpath_selector = f"//{element.name}[contains(@class, {quoteattr(classes)})]"
        elements_found = driver.find_elements("xpath", xpath_selector)
        if len(elements_found) == 1:
            return xpath_selector

    return None


# read page objects
def read_page_objects_metadata(driver):

    html = driver.page_source.encode("utf-8")
    soup = BeautifulSoup(html, 'html.parser')
    elements = soup.find_all(['input', 'button', 'a', 'img'])

    counter = defaultdict(int)
    result = defaultdict(list)

    for element in elements:
        tag = element.name
        selector = generate_selector(element, driver)
        key = generate_key(tag, element, counter)

        result[tag].append({
            "key": key,
            "selector": selector,
            "attributes": {attr: element.get(attr) for attr in element.attrs},
            "text": element.get_text(strip=True),
            "visibility": "visible" if element.get('type') != 'hidden' else "invisible"
        })
    return result

print("Done!")

### Logger

In [ ]:
def instrument_logs(driver, server_url="http://localhost:8000"):
    javascript_code = f"""
    window.lastProcessedEvent = {{}};
    window.clickedElements = new Set();
    window.initialValues = {{}};
    window.enterPressed = {{}};

    function sendLogToServer(logData) {{
        fetch("http://localhost:8000", {{
            method: 'POST',
            headers: {{
                'Content-Type': 'application/json'
            }},
            body: JSON.stringify(logData),
            mode: 'no-cors'  // Adicionar modo no-cors
        }});
    }}

    function clickListener(event) {{
        let element = event.target;
        let tagName = element.tagName.toLowerCase();
        let key = element.getAttribute('key');
        if (key === null || key === undefined || window.clickedElements.has(element)) return;

        let detail = tagName === 'button' ? 'text: ' + element.innerText : 'value: ' + element.value;
        sendLogToServer({{type: 'click', tagName: tagName, key: key, detail: detail}});
        window.clickedElements.add(element);
        window.initialValues[key] = element.value;
        window.enterPressed[key] = false;
    }}

    function keydownListener(event) {{
        let element = document.activeElement;
        let tagName = element.tagName.toLowerCase();
        let key = element.getAttribute('key');
        if (key === null || key === undefined) return;

        let eventIdentifier = tagName + '_' + key + '_' + event.key;
        if (window.lastProcessedEvent[key] === eventIdentifier) return;

        if (event.key === 'Enter') {{
            sendLogToServer({{type: 'sendkey', tagName: tagName, key: key, text: element.value, specialKey: '<ENTER>'}});
            window.enterPressed[key] = true;
        }}

        window.lastProcessedEvent[key] = eventIdentifier;
    }}

    function blurListener(event) {{
        let element = event.target;
        let key = element.getAttribute('key');
        if (key === null || key === undefined || window.enterPressed[key]) return;

        let text = element.value;
        if (text && window.initialValues[key] !== text) {{
            sendLogToServer({{type: 'sendkey', tagName: tagName, key: key, text: text}});
        }}
        window.initialValues[key] = text;
    }}

    if (window.hasEventListeners) {{
        document.removeEventListener('click', clickListener);
        document.removeEventListener('keydown', keydownListener);
        document.removeEventListener('blur', blurListener, true);
    }}

    window.hasEventListeners = true;
    document.addEventListener('click', clickListener);
    document.addEventListener('keydown', keydownListener);
    document.addEventListener('blur', blurListener, true);
    """
    driver.execute_script(javascript_code)


global browser_event_logger
browser_event_logger = None

class LoggingHTTPRequestHandler(http.server.SimpleHTTPRequestHandler):
    def log_message(self, format, *args):
        return

    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        post_data = self.rfile.read(content_length)

        if(browser_event_logger is None):
            print(post_data.decode('utf-8'))
        else:
            browser_event_logger(post_data.decode('utf-8'))

        self.send_response(200)
        self.send_header('Access-Control-Allow-Origin', '*')
        self.end_headers()


def start_server(port=8000):
    with socketserver.TCPServer(("", port), LoggingHTTPRequestHandler) as httpd:
        print(f"Serving at port {port}")
        httpd.serve_forever()


### GUI Widgets

In [ ]:

class MyDataFrame(pandas.DataFrame):
    def __finalize__(self, other, method=None, **kwargs):
        return super().__finalize__(other, method, **kwargs)

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        try: self.change_listener(None)
        except: pass


    @property
    def loc(self):
        loc_idx = super().loc

        class LocIndexer:
            def __getitem__(self, idx):
                return loc_idx.__getitem__(idx)
            
            def __setitem__(self, idx, value):
                loc_idx.__setitem__(idx, value)
                try: self.change_listener(None)
                except: pass          

        result = LocIndexer()
        result.change_listener = self.change_listener
        return result

    def set_change_listener(self, listener):
        self.change_listener = listener


class Table:
    def __init__(self, df=None, change_data_listenert=None):
        self.content = widgets.HBox()

        self.data_event = None
        self.web_event = None
        self.fire_change_data_event = change_data_listenert

        self.data = df if df is not None else MyDataFrame(
            columns=["Alias", "Text", "Web", "Export", "Data", "Type", "Att"])
        self.reload()

    def try_fire_data_event(self, b, index):
        if self.data_event is not None:
            dt = self.data["Data"][index]
            self.data_event(dt)

    def try_fire_web_event(self, b, index):
        dt = self.data["Data"][index]
        self.data_event(dt)
        
        global test_window
        highlight_element(test_window, self.data["Web"][index])

    def reload(self, df=None):

        if df is not None: self.data = df
        df = self.data
        lines = len(self.data) + 1
        grid = widgets.GridspecLayout(lines, 2)

        for i in range(lines):
            if i == 0: 
                grid[i, 0] = widgets.HTML(value="<b>Alias</b>")
                continue
            
            cell_value_alias = getattr(df.iloc[i-1], "Alias", '') if i > 0 else ''
            cell_value_web = getattr(df.iloc[i-1], "Web", '') if i > 0 else ''

            pnl = widgets.Button(tooltip=cell_value_web, icon='search', layout=widgets.Layout(width='50px', height='30px'))
            pnl.on_click(lambda b, index=i-1: self.try_fire_web_event(b, index))

            grid[i, 0] = widgets.Text(value=str(cell_value_alias), layout=widgets.Layout(width='250px', weight='2'))        
            grid[i, 1] = pnl

        self.content.children = [grid]


class Properties:
    
    def __init__(self, properties={}):        
        self.content = widgets.HBox()
        self.reload(properties)

    def reload(self, properties={}):
        num_props = len(properties)
        grid = widgets.GridspecLayout(num_props + 1, 2, width='auto')
        grid[0, 0] = widgets.HTML(value="<b>Property</b>", layout=widgets.Layout(width='80px'))
        grid[0, 1] = widgets.HTML(value="<b>Value</b>", layout=widgets.Layout(width='400px'))  

        i = 0
        for i, (key, value) in enumerate(properties.items(), start=1):
            grid[i, 0] = widgets.Text(value=key, layout=widgets.Layout(width='80px'), disabled=True, bold=True)
            grid[i, 1] = widgets.Text(value=str(value), layout=widgets.Layout(width='400px'))

        self.content.children = [grid]


def new_cell(content, width='100%', height='100px', scroll=False):
    return widgets.Box([content], layout=widgets.Layout(
        border='5px solid white',
        width=width,
        height=height,
        overflow='auto' if scroll else 'hidden'
    ))


class Window():

    def __init__(self, new_window_event, inspect_event, on_data_changed):
         
        _window = self
        
        self.new_window_event = new_window_event
        self.inspect_event = inspect_event
        self.on_data_changed = on_data_changed
        self.table = None

        dataframe = MyDataFrame(columns=["Alias", "Text", "Web", "Export", "Data", "Type", "Att"])
        
        # data changed event
        def try_fire_data_changed_event(self, b=None):
            _window.on_data_changed(_window)
        self.table = Table(dataframe, try_fire_data_changed_event)

        self.properties = Properties()
        self.try_fire_data_changed_event = try_fire_data_changed_event
        self.table.data.set_change_listener(self.try_fire_data_changed_event) 

        cell2 = new_cell(self.table.content, width='350px', height='250px', scroll=True)
        cell3 = new_cell(self.properties.content, width='500px', height='250px', scroll=True)
        
        self.dev_n_qa = new_cell(HTML(), width='350px', height='660px')
        self.page_objects = new_cell(HTML(), width='500px', height='660px', scroll=True)
        footer = widgets.HBox([self.dev_n_qa, self.page_objects])

        self.hbox = widgets.HBox([cell2, cell3], layout=widgets.Layout(height='300px'))
        self.content = widgets.VBox([self.hbox, footer], layout=widgets.Layout(height='auto'))             
        self.table.data_event = _window.properties.reload

    def set_dev_n_qa(self, readme):
        self.dev_n_qa.children = [HTML(value=readme)]

    def set_page_objects(self, lines):
        self.page_objects.children = [HTML(value="<br/><br/>\n".join(lines))]
    
    def redraw(self):
        clear_output(wait=True)
        display(self.content)

    def reload(self, df=None, props={}):
        if df is None: return
        self.table.reload(df)
        self.table.data.set_change_listener(self.try_fire_data_changed_event)
        self.try_fire_data_changed_event()

print("Done!")  

### GUI Events

In [ ]:
global test_window, logs
test_window=None
logs=[]


def log_event(window, event, reset=False):
    global test_window, logs
    if reset: logs=[]
    logs.append(event)
    window.set_page_objects(logs)


def new_window(wait=0):
    global test_window
    try: test_window.quit()
    except: test_window = None
    test_window = new_driver("https://life.stg.wellzesta.com/login")
    time.sleep(wait)


def data_changed(window=None):
    print("data changed")
    pass


def inspect_webpage(window=None, table=None, properties=None):
    global test_window, selectors
    metadata = None
    selectors = {}
    try:
        metadata = read_page_objects_metadata(test_window)
    except:
        new_window()
        return

    rows = []

    for tag_name, elements in metadata.items():
        for element in elements:

            att = element.pop('attributes', {})
            clazz =  att.pop('class', [])

            for k, v in att.items():
                element[k] = v

            element['class'] = clazz
            key = element.get('key', '')
            selector = element.get('selector', '')

            js = f"""
    var element = document.evaluate('{selector}', document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null).singleNodeValue;
    if (element) element.setAttribute('key','{key}');
            """
            
            rows.append({
                "Export": False,
                "Alias": key,
                "key": key,
                "Web": selector,
                "Type": tag_name,
                "Data": element
            })
            
            test_window.execute_script(js)
            selectors[key]=selector

    df = MyDataFrame(rows)
    table.reload(df)
    properties.reload({})

    #log_event(window, str(selectors))
    
    df.set_change_listener(window.try_fire_data_changed_event)
    window.try_fire_data_changed_event(window)

    instrument_logs(test_window)

README = """<i>
When a DEV loves a QA <br/>
Can't keep his mind on any code <br/>
He'd debug all the errors <br/>
For the quality she's known <br/>
 <br/>
Yes, she's tough, oh so demanding <br/>
But he's the champ, codes so well <br/>
Even writes a testing framework <br/>
Just to make her world excel <br/>
 <br/>
Turns his back on his patterns <br/>
If they don't pass her test <br/>
When a DEV loves a QA <br/>
He's precise in every line <br/>
Crafting codes she envisioned <br/>
 <br/>
He'd give up all his code-tricks <br/>
And embrace the cleaner code <br/>
If she said that's the way <br/>
It ought to be <br/>
 <br/>
Though she's strict, he's so adept <br/>
Together they're a perfect duet
</i>"""

class Playground:

    def __init__(self):

        window = Window(new_window, inspect_webpage, data_changed)
        window.set_dev_n_qa(README)
        window.redraw()

        server_thread = threading.Thread(target=start_server, args=(8000,))
        server_thread.daemon = True
        server_thread.start()
        clear_output()

        new_window()
        log_event(window,"{'framework':'selenium'}")

        global browser_event_logger

        def logger(event):
            log_event(window, event)

        browser_event_logger = logger

        def check_browser():
            print("check browser deamon")
            url_logged = False
            redraw = False
            global test_window
            while True:
                try:
                    if not is_page_loaded():
                        url_logged = False
                        print(f"loading {test_window.current_url}")
                        time.sleep(0.5)
                        continue

                    if not url_logged:
                        log_event(window,"{'loaded_url':'" + test_window.current_url + "'}")
                        url_logged = True
                except:
                    new_window()
                    time.sleep(1)
                    continue

                if not is_logging_instrumented():
                    window.redraw()
                    print(f"instrumenting {test_window.current_url}")
                    inspect_webpage(window, window.table, window.properties)            
                    time.sleep(5)
                    redraw = True
                    continue
                
                if redraw:
                    clear_output()
                    window.redraw()
                    redraw = False
                    
                time.sleep(0.5)

        browser_thread = threading.Thread(target=check_browser)
        browser_thread.daemon = True
        browser_thread.start()

print("Done!")

## Playground

In [ ]:
Playground() 